In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/workshops/District to SSO master.csv
/kaggle/input/workshops/WS location details.csv
/kaggle/input/workshops/all_region_workshops.csv
/kaggle/input/mayforheatmap/May.parquet


In [2]:
!pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.3 MB/s eta 0:00:00


In [3]:
df_tele = pd.read_parquet("/kaggle/input/mayforheatmap/May.parquet",engine='fastparquet')

In [4]:
df_tele.head()

,vehicleId,geohash_7,geohash_5,Latitude,Longitude,month,LOB,PL,PPL,region,state,city,rashturning,overspeeding,harshacceleration,harshbrake,VinCount,event_type,totalVinCount,eventdate_1
0,MAT790005L1J03687,tu7p9m8,tu7p9,25.255508,83.024368,5,MCV Trucks,LPT 1918,MCV LPT 19,North,Haryana,GURGAON,1,0,0,0,1,rashturning_event,1097,2024-05-01
1,MAT797031L2K05106,tg22ctf,tg22c,18.445358,79.170914,5,HCV Const,SIGNA 2821.K. FBV,MAV Tippers 28,South,Telangana,HYDERABAD,1,0,0,0,1,rashturning_event,95,2024-05-01
2,MAT790207NAJ09776,tukn0s1,tukn0,24.983597,84.399033,5,MCV Trucks,LPT1918 5L,MCV LPT 19,North,Haryana,GURGAON,1,0,0,0,1,rashturning_event,740,2024-05-01
3,MAT812033N5B03743,ttp45w5,ttp45,28.510208,77.502365,5,ICV Trucks,1109g LPT,ICV 10T - 12T,North,Uttar Pradesh,NOIDA,1,0,0,0,1,rashturning_event,357,2024-05-01
4,MAT563010N5N13935,ttr8nsh,ttr8n,29.553909,78.338699,5,ICV Trucks,1512 LPT,ICV 15T - 16T,North,Uttar Pradesh,BAREILLY,1,0,0,0,1,rashturning_event,381,2024-05-01


In [5]:
grouped_df = df_tele.groupby(['geohash_7', 'geohash_5', 'Latitude', 'Longitude', 'month', 'LOB', 'PL', 'PPL']).size().reset_index(name='vehicleId_count')

In [6]:
grouped_df.shape

(5017561, 9)

In [7]:
df_tel = grouped_df.query("vehicleId_count >=10")

In [8]:
df_tel.shape

(34929, 9)

In [9]:
df_workshop=pd.read_csv("/kaggle/input/workshops/all_region_workshops.csv")

In [10]:
df_workshop.head()

,Division,Lattitude,Longitude,SSO,State
0,100B520-Sv&Pa-Nagercoils-AfjTor,8.190837,77.419204,Salem,Tamil Nadu
1,1001650-Sv&Pa-Nagercoil-ABTInM,8.198264,77.431443,Salem,Tamil Nadu
2,2088645-Sv&Pa-Neyyattinkara-KdmMot,8.368100,77.082080,Cochin,Kerala
3,2089374-Sv&Pa-Vallioor-SavAut,8.370961,77.601832,Salem,Tamil Nadu
4,2088731-Sv&Pa-Trivandrum-VjyMtr,8.424550,76.962980,Cochin,Kerala


In [11]:
df_workshop = df_workshop.dropna()

In [12]:
df_workshop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1586 entries, 0 to 1590
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Division   1586 non-null   object 
 1   Lattitude  1586 non-null   float64
 2   Longitude  1586 non-null   float64
 3   SSO        1586 non-null   object 
 4   State      1586 non-null   object 
dtypes: float64(2), object(3)
memory usage: 74.3+ KB


In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances

# Function to calculate haversine distance matrix in kilometers
def haversine_distance_matrix(workshops, telemetry):
    # Convert degrees to radians
    workshops_rad = np.radians(workshops)
    telemetry_rad = np.radians(telemetry)

    # Calculate the pairwise haversine distance matrix
    distances = pairwise_distances(telemetry_rad, workshops_rad, metric='haversine')
    # Convert radians to kilometers (earth radius ~6371 km)
    distances_km = distances * 6371
    return distances_km
'''

# Sample dataframes
# Workshop dataframe
workshops = pd.DataFrame({
    'lat': [12.9715987, 13.0826802, 12.2958104],
    'lon': [77.5945627, 80.2707184, 76.6393805]
})

# Telemetry dataframe
telemetry = pd.DataFrame({
    'lat': [12.9715987, 13.0826802, 11.0168445],
    'lon': [77.5945627, 80.2707184, 76.9558321]
})
'''
# Calculate the haversine distance matrix in kilometers
distances_km = haversine_distance_matrix(df_workshop[['Lattitude', 'Longitude']], df_tel[['Latitude', 'Longitude']])

# Function to classify points based on distance
def classify_points(workshops, telemetry, distances_km, threshold_km=15):
    min_distances = distances_km.min(axis=1)
    colors = ['green' if dist <= threshold_km else 'red' for dist in min_distances]
    result = telemetry.copy()
    result['distance'] = min_distances
    result['color'] = colors
    return result

# Classify the telemetry points
classified_points = classify_points(df_workshop, df_tel, distances_km)
#print(classified_points)


In [14]:
classified_points.head()

,geohash_7,geohash_5,Latitude,Longitude,month,LOB,PL,PPL,vehicleId_count,distance,color
242,t9qygyn,t9qyg,8.251419,77.165909,5,ICV Trucks,912 LPK,ICV Tipper,37,15.918548,red
480,t9qyuc7,t9qyu,8.225327,77.205734,5,LCV,610 SK,LCV Tippers,10,20.908218,red
567,t9qyuss,t9qyu,8.243179,77.196121,5,LCV,610 SK,LCV Tippers,22,18.718955,red
568,t9qyusu,t9qyu,8.244553,77.196121,5,LCV,610 SK,LCV Tippers,12,18.605943,red
1313,t9qz5f0,t9qz5,8.273392,77.157669,5,LCV,710 SFC,LCV 7T,12,13.419008,green


In [15]:
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio


# Filtering vehicles based on distance from the workshop
vehicles_near = classified_points[classified_points['distance'] < 10]
vehicles_far = classified_points[classified_points['distance'] >= 10]

# Define the figure
fig = go.Figure()

# Adding vehicle locations (distance < 15 kms) in green with transparency
fig.add_trace(go.Scattermapbox(
    lat=vehicles_near['Latitude'],
    lon=vehicles_near['Longitude'],
    mode='markers',
    marker=dict(color='green', size=15, opacity=0.5),
    name="Vehicles < 15Km"
))

# Adding vehicle locations (distance >= 15 kms) in red with transparency
fig.add_trace(go.Scattermapbox(
    lat=vehicles_far['Latitude'],
    lon=vehicles_far['Longitude'],
    mode='markers',
    marker=dict(color='red', size=15, opacity=0.5),
    name="Vehicles >= 15Km"
))

# Adding workshop locations in blue with transparency
fig.add_trace(go.Scattermapbox(
    lat=df_workshop['Lattitude'],
    lon=df_workshop['Longitude'],
    mode='markers',
    marker=dict(color='blue', size=15, opacity=0.5),
    name="Workshops"
))

# Update the layout of the plot to use a mapbox style and set the center and zoom level for India
fig.update_layout(
    mapbox_style='open-street-map', 
    mapbox_center_lon=78.9629,  # Approximate center longitude of India
    mapbox_center_lat=20.5937,  # Approximate center latitude of India
    mapbox_zoom=4,  # Zoom level for India
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

# Save the plot as an HTML file
pio.write_html(fig, file='map_plot.html', auto_open=True)

# Show the plot
#fig.show()


In [16]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np


# Function to generate points of a circle
def generate_circle_points(lat, lon, radius_km, num_points=100):
    """
    Generate latitude and longitude points that form a circle around a central point.
    """
    R = 6371  # Radius of the Earth in km
    d = radius_km / R  # Angular distance in radians

    lat, lon = np.radians(lat), np.radians(lon)
    circle_lats, circle_lons = [], []
    for i in range(num_points):
        theta = 2 * np.pi * i / num_points
        lat_i = np.arcsin(np.sin(lat) * np.cos(d) + np.cos(lat) * np.sin(d) * np.cos(theta))
        lon_i = lon + np.arctan2(np.sin(theta) * np.sin(d) * np.cos(lat), np.cos(d) - np.sin(lat) * np.sin(lat_i))
        circle_lats.append(np.degrees(lat_i))
        circle_lons.append(np.degrees(lon_i))
    return circle_lats, circle_lons

# Generate circle points
center_lat, center_lon = 12.9715987, 77.5945627  # Center point (e.g., one of the workshops)
circle_radius_km = 10  # Radius of the circle in km
circle_lats, circle_lons = generate_circle_points(center_lat, center_lon, circle_radius_km)

# Define the figure
fig = go.Figure()

# Adding heatmap trace
fig.add_trace(go.Densitymapbox(
    lat=classified_points['Latitude'],
    lon=classified_points['Longitude'],
    radius=30,  # Adjust radius for kernel density estimation
    colorscale='YlOrRd',
    zmax=10,
    zmin=0,
    name='Density Heatmap'
))

# Adding vehicle locations (distance < 15 kms) in green with transparency
fig.add_trace(go.Scattermapbox(
    lat=classified_points['Latitude'],
    lon=classified_points['Longitude'],
    mode='markers',
    marker=dict(color='black', size=7, opacity=0.7),
    name="Vehicles"
))

# Adding workshop locations in blue with transparency
fig.add_trace(go.Scattermapbox(
    lat=df_workshop['Lattitude'],
    lon=df_workshop['Longitude'],
    mode='markers',
    marker=dict(color='blue', size=15, opacity=0.7),
    name="Workshops"
))

# Adding circle trace
fig.add_trace(go.Scattermapbox(
    lat=circle_lats,
    lon=circle_lons,
    mode='lines',
    line=dict(color='orange', width=2),
    name=f"Circle with {circle_radius_km}Km radius"
))

# Update the layout of the plot to use a mapbox style and set the center and zoom level for India
fig.update_layout(
    mapbox_style='open-street-map', 
    mapbox_center_lon=78.9629,  # Approximate center longitude of India
    mapbox_center_lat=20.5937,  # Approximate center latitude of India
    mapbox_zoom=5,  # Zoom level for India
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

# Save the plot as an HTML file
import plotly.io as pio
#pio.write_html(fig, file='map_plot.html', auto_open=True)

# Show the plot
#fig.show()


In [17]:
import plotly.graph_objects as go

# Filtering vehicles based on distance from the workshop
vehicles_near = classified_points[classified_points['distance'] < 15]
vehicles_far = classified_points[classified_points['distance'] >= 15]

# Define the figure
fig = go.Figure()

# Adding heatmap trace
fig.add_trace(go.Densitymapbox(
    lat=classified_points['Latitude'],
    lon=classified_points['Longitude'],
    radius=30,  # Adjust radius for kernel density estimation
    colorscale=[[0,"green"], [0.5, "yellow"], [1, "red"]],  # Red, green, yellow colors
    zmax=10,
    zmin=2,
    name='Density Heatmap'
))

# Adding vehicle locations (distance < 15 kms) in green with transparency
fig.add_trace(go.Scattermapbox(
    lat=vehicles_near['Latitude'],
    lon=vehicles_near['Longitude'],
    mode='markers',
    marker=dict(color='green', size=15, opacity=0.5),
    name="Vehicles < 15Km"
))

# Adding vehicle locations (distance >= 15 kms) in red with transparency
fig.add_trace(go.Scattermapbox(
    lat=vehicles_far['Latitude'],
    lon=vehicles_far['Longitude'],
    mode='markers',
    marker=dict(color='red', size=15, opacity=0.5),
    name="Vehicles >= 15Km"
))

# Adding workshop locations in blue with transparency
fig.add_trace(go.Scattermapbox(
    lat=df_workshop['Lattitude'],
    lon=df_workshop['Longitude'],
    mode='markers',
    marker=dict(color='blue', size=15, opacity=1),
    name="Workshops"
))

# Update the layout of the plot to use a mapbox style and set the center and zoom level for India
fig.update_layout(
    mapbox_style='open-street-map', 
    mapbox_center_lon=78.9629,  # Approximate center longitude of India
    mapbox_center_lat=20.5937,  # Approximate center latitude of India
    mapbox_zoom=5,  # Zoom level for India
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

# Save the plot as an HTML file
import plotly.io as pio
pio.write_html(fig, file='map_plot.html', auto_open=True)

# Show the plot
fig.show()


In [18]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Sample data for demonstration (replace this with your actual data)

# Filtering vehicles based on distance from the workshop
vehicles_near = classified_points[classified_points['distance'] < 15]
vehicles_far = classified_points[classified_points['distance'] >= 15]

# Define the figure
fig = go.Figure()

# Adding heatmap trace
fig.add_trace(go.Densitymapbox(
    lat=classified_points['Latitude'],
    lon=classified_points['Longitude'],
    radius=10,  # Smaller radius for more detailed heatmap
    colorscale=[[0, "green"], [0.5, "yellow"], [1, "red"]],  # Red, green, yellow colors
    zmax=1,
    zmin=0,
    opacity=0.6,
    name='Density Heatmap'
))

# Adding vehicle locations (distance < 15 kms) in green with transparency
fig.add_trace(go.Scattermapbox(
    lat=vehicles_near['Latitude'],
    lon=vehicles_near['Longitude'],
    mode='markers',
    marker=dict(color='green', size=15, opacity=0.5),
    text=vehicles_near['vehicleId_count'],
    hoverinfo='text',
    name="Vehicles < 15Km"
))

# Adding vehicle locations (distance >= 15 kms) in red with transparency
fig.add_trace(go.Scattermapbox(
    lat=vehicles_far['Latitude'],
    lon=vehicles_far['Longitude'],
    mode='markers',
    marker=dict(color='red', size=15, opacity=0.5),
    text=vehicles_far['vehicleId_count'],
    hoverinfo='text',
    name="Vehicles >= 15Km"
))

# Adding workshop locations in blue with transparency
fig.add_trace(go.Scattermapbox(
    lat=df_workshop['Lattitude'],
    lon=df_workshop['Longitude'],
    mode='markers',
    marker=dict(color='blue', size=15, opacity=0.5),
    name="Workshops"
))

# Update the layout of the plot to use a mapbox style and set the center and zoom level for India
fig.update_layout(
    mapbox_style='open-street-map', 
    mapbox_center_lon=78.9629,  # Approximate center longitude of India
    mapbox_center_lat=20.5937,  # Approximate center latitude of India
    mapbox_zoom=5,  # Zoom level for India
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

# Save the plot as an HTML file
import plotly.io as pio
pio.write_html(fig, file='map_plot.html', auto_open=True)

# Show the plot
fig.show()
